In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import nltk 
import re
from sklearn.feature_selection import chi2
import numpy as np
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk import tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
#rom sklearn.feature_extraction.text import CountVectorizer
#from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.datasets import load_files
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pipelines

In [2]:
df=pd.read_csv("fullReviews.csv")
df


,Unnamed: 0,Unnamed: 0.1,Accomodation/Infrastructure,Faculty,Other,Placements,Rating,Review,Social,Title,Website
0,0,0,I do agree with nupur that Phd is for full tim...,"So, I can assure once you do your research wor...",NaN,NaN,90.0,I am a PhD student and about to submit my thes...,NaN,Focus on quality work,career
1,1,1,NaN,Jaypee is one of the best engineering college ...,NaN,Highest package is 48 lakh in 2017 and average...,100.0,Jaypee is one of the best engineering college ...,NaN,My view on jaypee,career
2,2,2,NaN,NaN,NaN,jaypee is a excellent college for computer sci...,100.0,jaypee is a excellent college for computer sci...,NaN,Best private college in North india,career
3,3,3,"it was situated in the heart of NCR, the campu...","Our faculty is so good they teach well, scorin...",NaN,NaN,80.0,The infrastructure of the college is good. it ...,NaN,My words on jaypee institute of information te...,career
4,4,4,Also they have made use of available space wel...,Professors are mostly doctorate holders and se...,NaN,But its all right once you get a hang of it.Ov...,90.0,Jaypee sec 62 is a decent college with student...,NaN,Highly competitive Environment,career
...,...,...,...,...,...,...,...,...,...,...,...
823,823,823,Gender ratio is almost equal which is quite un...,Their was no stipend .,AIEEEI worked at DUCAT . I worked and was trai...,Over 50 companies came last year .Campus life ...,82.0,AIEEEThe best thing about jaypee is its place...,Campus life is very good here in jaypee. Gend...,NaN,collegedunia.com
824,824,824,And a few more celebrations.No I have not avai...,"IIT JEE, I was interested in electronics and ...","IIT JEE, I was interested in electronics and c...",Like the college held dandiya nights during na...,77.0,"IIT JEE, I was interested in electronics and ...","Gender ratio is 3:2. There are more boys, just...",NaN,collegedunia.com
825,825,825,But there is no loan facility that is offered ...,NaN,JIIT admits students solely through JEE Mains....,Another scholarship Meriam-Webster scholarship...,85.0,JIIT admits students solely through JEE Mains...,NaN,NaN,collegedunia.com
826,826,826,As it was my second unsuccessful attempt to J...,You can expect to come across all your basic c...,As it was my second unsuccessful attempt to JE...,Offered to those who secure 7.0+ GPA in first ...,77.0,As it was my second unsuccessful attempt to J...,NaN,NaN,collegedunia.com


In [3]:
def partition(x):
    if x>=0 and x<33:
        return 'negative'
    elif x>=33 and x<65:
        return 'neutral'
    elif x>=65:
        return "positive"

In [4]:
score=df['Rating']
positivenegative=[]
ls=list(score)
for i in ls:
    c=int(i)
    ans=partition(c)
    positivenegative.append(ans)
df['Roundoff']=positivenegative

In [5]:
df['Roundoff'].value_counts()

positive    751
neutral      62
negative     15
Name: Roundoff, dtype: int64

In [6]:
from io import StringIO
col = ['Roundoff', 'Review']
df = df[col]
df = df[pd.notnull(df['Review'])]
df.columns = ['Roundoff', 'Review']
df['rating_id'] = df['Roundoff'].factorize()[0]
rating_id_df = df[['Roundoff', 'rating_id']].drop_duplicates().sort_values('rating_id')
rating_to_id = dict(rating_id_df.values)
id_to_rating = dict(rating_id_df[['rating_id', 'Roundoff']].values)


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(df.Review).toarray()
labels = df.rating_id
features.shape

(828, 3870)

In [8]:
N = 2
for Roundoff, rating_id in sorted(rating_to_id.items()):
    features_chi2 = chi2(features, labels == rating_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    trigrams = [v for v in feature_names if len(v.split(' ')) == 3]
    print("# '{}':".format(Roundoff))
    print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-N:])))
    print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-N:])))
    print("  . Most correlated trigrams:\n. {}".format('\n. '.join(trigrams[-N:])))

# 'negative':
  . Most correlated unigrams:
. combined
. kept
  . Most correlated bigrams:
. amazon visit
. nice college
  . Most correlated trigrams:
. 
# 'neutral':
  . Most correlated unigrams:
. met
. upgrade
  . Most correlated bigrams:
. great experience
. good supportive
  . Most correlated trigrams:
. 
# 'positive':
  . Most correlated unigrams:
. met
. upgrade
  . Most correlated bigrams:
. great experience
. good supportive
  . Most correlated trigrams:
. 


In [117]:
x=[]
col=df["Review"]
for text in col:
    x.append(text)
y=[]
col=df["Roundoff"]
for text in col:
    y.append(text)

In [118]:
X_train, X_test, y_train, y_test = train_test_split(df['Review'], df['Roundoff'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)
clf2 = RandomForestClassifier().fit(X_train_tfidf, y_train)
clf3=LogisticRegression().fit(X_train_tfidf, y_train)
clf4= LinearSVC().fit(X_train_tfidf, y_train)

In [119]:
y_predic=clf.predict(count_vect.transform(x))#naive-bayes
y_predic2=clf2.predict(count_vect.transform(x))#random-forest
y_predic3=clf3.predict(count_vect.transform(x))#logistic regression
y_predic4=clf4.predict(count_vect.transform(x)) #linear svc

In [120]:
results = confusion_matrix(y,y_predic)
print(results)#naive-bayes

[[  0   0  15]
 [  0   0  62]
 [  0   0 751]]


In [121]:
results = confusion_matrix(y,y_predic2)
print(results) #random forest

[[  6   3   6]
 [  0  56   6]
 [  1 230 520]]


In [122]:
results = confusion_matrix(y,y_predic3)
print(results) #logistic regression

[[  0   0  15]
 [  0  14  48]
 [  0   0 751]]


In [123]:
results4 = confusion_matrix(y,y_predic4)
print(results) #linear svc

[[  0   0  15]
 [  0  14  48]
 [  0   0 751]]


In [124]:
models = [clf,clf2,clf3,clf4]
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

In [125]:
cv_df.groupby('model_name').accuracy.mean()

model_name
LinearSVC                 0.962526
LogisticRegression        0.907016
MultinomialNB             0.907016
RandomForestClassifier    0.948068
Name: accuracy, dtype: float64

In [178]:
col=df["Faculty"]
for i in range(0,828):
    text=df["Faculty"].values[i]
    if text==None:
        df["Facukty"].values[i]="Unkown"

In [182]:

df.fillna("Unknown", inplace = True) 


In [205]:
lis1=[]
for i in range(0,828):
    text=df["Faculty"].values[i]
    if text!="Unknown":
        ansf=clf4.predict(count_vect.transform([text]))
        lis1.append(ansf)
    else:
        lis1.append("Not known")
lis1
    

[array(['positive'], dtype=object),
 array(['positive'], dtype=object),
 'Not known',
 array(['positive'], dtype=object),
 array(['positive'], dtype=object),
 'Not known',
 array(['positive'], dtype=object),
 array(['positive'], dtype=object),
 array(['positive'], dtype=object),
 array(['positive'], dtype=object),
 array(['neutral'], dtype=object),
 array(['positive'], dtype=object),
 array(['neutral'], dtype=object),
 array(['neutral'], dtype=object),
 array(['positive'], dtype=object),
 'Not known',
 array(['positive'], dtype=object),
 array(['positive'], dtype=object),
 array(['positive'], dtype=object),
 array(['positive'], dtype=object),
 'Not known',
 array(['neutral'], dtype=object),
 array(['positive'], dtype=object),
 array(['positive'], dtype=object),
 array(['neutral'], dtype=object),
 array(['neutral'], dtype=object),
 array(['positive'], dtype=object),
 array(['positive'], dtype=object),
 array(['positive'], dtype=object),
 array(['neutral'], dtype=object),
 'Not known',
 

In [219]:
lis2=[]
for i in range(0,828):
    text=df["Accomodation/Infrastructure"].values[i]
    if text!="Unknown":
        ansa=clf4.predict(count_vect.transform([text]))
        lis2.append(ansa)
    else:
        lis2.append("Not Known")

In [218]:
lis3=[]
for i in range(0,828):
    text=df["Placements"].values[i]
    if text!="Unkown":
        ansp=clf4.predict(count_vect.transform([text]))
        lis3.append(ansp)
    else:
        lis3.append("Not Known")


In [224]:
lis4=[]
for i in range(0,828):
    text=df["Social"].values[i]
    anss=clf4.predict(count_vect.transform([text]))
    if text!="Unknown":
        lis4.append(anss)
    else:
        lis4.append("Not known")

In [225]:
lis5=[]
for i in range(0,828):
    text=df["Other"].values[i]
    anso=clf4.predict(count_vect.transform([text]))
    if text!="Unknown":
        lis5.append(anso)
    else:
        lis5.append("Unknown")

In [233]:
count1=0
for i in range(0,828):
    roundoff=lis1[i]
    #print(roundoff)
    if roundoff=="positive":
        count1=count1+1
print(count1)
rating_faculty=(count1/828)*100
rating_faculty


610


73.67149758454107

In [235]:
count2=0
for i in range(0,828):
    roundoff=lis2[i]
    #print(roundoff)
    if roundoff=="positive":
        count2=count2+1
print(count2)
rating_accomadation=(count2/828)*100
rating_accomadation


686


82.85024154589372

In [237]:
count3=0
for i in range(0,828):
    roundoff=lis3[i]
    #print(roundoff)
    if roundoff=="positive":
        count3=count3+1
print(count3)
rating_placements=(count3/828)*100
rating_placements


723


87.31884057971014

In [239]:
count4=0
for i in range(0,828):
    roundoff=lis4[i]
    #print(roundoff)
    if roundoff=="positive":
        count4=count4+1
print(count4)
rating_social=(count4/828)*100
rating_social


61


7.367149758454106

In [240]:
count5=0
for i in range(0,828):
    roundoff=lis5[i]
    #print(roundoff)
    if roundoff=="positive":
        count5=count5+1
print(count5)
rating_other=(count5/828)*100
rating_other


527


63.64734299516909

In [ ]:
def segregate(query):
    